In [1]:
!pip install -q segmentation-models-pytorch timm albumentations

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.8/154.8 kB 2.3 MB/s eta 0:00:00:00:01


In [2]:
import os
import cv2
import numpy as np
import torch
import random
import albumentations as A
import segmentation_models_pytorch as smp

from torch.utils.data import Dataset, DataLoader
from albumentations.pytorch import ToTensorV2
from tqdm import tqdm

In [3]:
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

set_seed()

In [4]:
TRAIN_IMG = "/kaggle/input/datasets/arunkumarkorra/hwi-dataset/Offroad_Segmentation_Training_Dataset/Offroad_Segmentation_Training_Dataset/train/Color_Images"
TRAIN_MASK = "/kaggle/input/datasets/arunkumarkorra/hwi-dataset/Offroad_Segmentation_Training_Dataset/Offroad_Segmentation_Training_Dataset/train/Segmentation"

VAL_IMG = "/kaggle/input/datasets/arunkumarkorra/hwi-dataset/Offroad_Segmentation_Training_Dataset/Offroad_Segmentation_Training_Dataset/val/Color_Images"
VAL_MASK = "/kaggle/input/datasets/arunkumarkorra/hwi-dataset/Offroad_Segmentation_Training_Dataset/Offroad_Segmentation_Training_Dataset/val/Segmentation"

TEST_IMG = "/kaggle/input/datasets/arunkumarkorra/hwi-dataset/Offroad_Segmentation_testImages/Offroad_Segmentation_testImages/Color_Images"

In [5]:
value_map = {
    0: 0,
    100: 1,
    200: 2,
    300: 3,
    500: 4,
    550: 5,
    700: 6,
    800: 7,
    7100: 8,
    10000: 9
}

NUM_CLASSES = 10

In [6]:
class OffroadDataset(Dataset):
    def __init__(self, img_dir, mask_dir=None, transform=None):
        self.img_dir = img_dir
        self.mask_dir = mask_dir
        self.images = sorted(os.listdir(img_dir))
        self.transform = transform

    def convert_mask(self, mask):
        new_mask = np.zeros_like(mask)
        for raw, new in value_map.items():
            new_mask[mask == raw] = new
        return new_mask

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_name = self.images[idx]
        img_path = os.path.join(self.img_dir, img_name)

        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        if self.mask_dir:
            mask_path = os.path.join(self.mask_dir, img_name)
            mask = cv2.imread(mask_path, cv2.IMREAD_UNCHANGED)
            mask = self.convert_mask(mask)

            augmented = self.transform(image=image, mask=mask)
            image = augmented["image"]
            mask = augmented["mask"].long()

            return image, mask
        else:
            augmented = self.transform(image=image)
            image = augmented["image"]
            return image, img_name

In [7]:
train_transform = A.Compose([
    A.Resize(512, 512),
    A.HorizontalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.4),
    A.HueSaturationValue(p=0.3),
    A.GaussianBlur(p=0.2),
    A.Normalize(),
    ToTensorV2()
])

val_transform = A.Compose([
    A.Resize(512, 512),
    A.Normalize(),
    ToTensorV2()
])

In [8]:
train_dataset = OffroadDataset(TRAIN_IMG, TRAIN_MASK, train_transform)
val_dataset = OffroadDataset(VAL_IMG, VAL_MASK, val_transform)

train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=4, shuffle=False, num_workers=2)

In [9]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = smp.Segformer(
    encoder_name="mit_b3",
    encoder_weights="imagenet",
    in_channels=3,
    classes=NUM_CLASSES
)

model.to(device)

config.json:   0%|          | 0.00/135 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/178M [00:00<?, ?B/s]

Segformer(
  (encoder): MixVisionTransformerEncoder(
    (patch_embed1): OverlapPatchEmbed(
      (proj): Conv2d(3, 64, kernel_size=(7, 7), stride=(4, 4), padding=(3, 3))
      (norm): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
    )
    (patch_embed2): OverlapPatchEmbed(
      (proj): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (norm): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
    )
    (patch_embed3): OverlapPatchEmbed(
      (proj): Conv2d(128, 320, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (norm): LayerNorm((320,), eps=1e-05, elementwise_affine=True)
    )
    (patch_embed4): OverlapPatchEmbed(
      (proj): Conv2d(320, 512, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
    )
    (block1): Sequential(
      (0): Block(
        (norm1): LayerNorm((64,), eps=1e-06, elementwise_affine=True)
        (attn): Attention(
          (q): Linear(in_featur

In [10]:
loss_fn = smp.losses.DiceLoss(mode="multiclass") + \
          smp.losses.SoftCrossEntropyLoss()

optimizer = torch.optim.AdamW(model.parameters(), lr=3e-4, weight_decay=1e-4)

scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
    optimizer, T_max=40
)

TypeError: unsupported operand type(s) for +: 'DiceLoss' and 'SoftCrossEntropyLoss'

In [ ]:
def compute_iou(pred, mask):
    pred = torch.argmax(pred, dim=1)

    ious = []
    for cls in range(NUM_CLASSES):
        intersection = ((pred == cls) & (mask == cls)).sum().item()
        union = ((pred == cls) | (mask == cls)).sum().item()
        if union == 0:
            continue
        ious.append(intersection / union)
    return np.mean(ious)

In [ ]:
best_iou = 0
EPOCHS = 40

for epoch in range(EPOCHS):
    model.train()
    train_loss = 0

    for imgs, masks in tqdm(train_loader):
        imgs, masks = imgs.to(device), masks.to(device)

        preds = model(imgs)
        loss = loss_fn(preds, masks)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

    scheduler.step()

    # Validation
    model.eval()
    val_iou = []

    with torch.no_grad():
        for imgs, masks in val_loader:
            imgs, masks = imgs.to(device), masks.to(device)
            preds = model(imgs)
            val_iou.append(compute_iou(preds, masks))

    mean_iou = np.mean(val_iou)

    print(f"Epoch {epoch+1}/{EPOCHS} | Loss: {train_loss/len(train_loader):.4f} | Val IoU: {mean_iou:.4f}")

    if mean_iou > best_iou:
        best_iou = mean_iou
        torch.save(model.state_dict(), "best_model.pth")
        print("Saved best model")